In [ ]:
import os
import json
import cv2
import numpy as np
import albumentations as A

# --- Configuration ---
# Update these paths to match your directory structure
ORIGINAL_DATASET_DIR = "/workspace/dataset/ppe_coco_format"
AUGMENTED_DATASET_DIR = "/workspace/dataset/ppe_coco_format_augmented"

ORIGINAL_ANNOTATION_FILE = os.path.join(ORIGINAL_DATASET_DIR, "annotations", "instances_train.json")
ORIGINAL_IMAGE_DIR = os.path.join(ORIGINAL_DATASET_DIR, "images", "train")

AUGMENTED_IMAGE_DIR = os.path.join(AUGMENTED_DATASET_DIR, "images", "train")
AUGMENTED_ANNOTATION_FILE = os.path.join(AUGMENTED_DATASET_DIR, "annotations", "instances_train.json")

# Number of times to augment each image (e.g., 1x, 2x, etc.)
# A value of 2 means each original image will be used to generate 2 new augmented images.
AUGMENTATION_FACTOR = 2

# --- Define Augmentation Pipeline ---
# This pipeline includes augmentations that affect lighting, color, and noise.
# Bounding box parameters are set to ensure they are handled correctly.
# We will apply this to all images for a more robust model.
augmentations = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.MotionBlur(p=0.5),
], bbox_params=A.BboxParams(format='coco', label_fields=['class_ids']))

def augment_dataset():
    """
    Reads a COCO dataset, applies augmentations, and saves a new augmented dataset.
    """
    print("Starting dataset augmentation process...")

    # Create necessary directories
    os.makedirs(AUGMENTED_IMAGE_DIR, exist_ok=True)
    os.makedirs(os.path.join(AUGMENTED_DATASET_DIR, "annotations"), exist_ok=True)

    # Load original COCO annotations
    with open(ORIGINAL_ANNOTATION_FILE, 'r') as f:
        coco_data = json.load(f)

    # Initialize new COCO data structure for the augmented dataset
    new_coco_data = {
        "info": coco_data["info"],
        "licenses": coco_data["licenses"],
        "images": [],
        "annotations": [],
        "categories": coco_data["categories"]
    }

    image_id_counter = 0
    annotation_id_counter = 0

    # Process original images and annotations
    for image_info in coco_data["images"]:
        image_path = os.path.join(ORIGINAL_IMAGE_DIR, image_info["file_name"])
        image = cv2.imread(image_path)

        # Get annotations for the current image
        image_annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] == image_info["id"]]

        # Prepare annotations for Albumentations (COCO format)
        bboxes = [ann["bbox"] for ann in image_annotations]
        class_ids = [ann["category_id"] for ann in image_annotations]

        # Add the original image and annotations to the new dataset
        new_coco_data["images"].append({
            "id": image_id_counter,
            "file_name": image_info["file_name"],
            "width": image_info["width"],
            "height": image_info["height"]
        })
        for ann in image_annotations:
            ann['id'] = annotation_id_counter
            ann['image_id'] = image_id_counter
            new_coco_data["annotations"].append(ann)
            annotation_id_counter += 1
        image_id_counter += 1

        # Apply augmentations AUGMENTATION_FACTOR times
        for i in range(AUGMENTATION_FACTOR):
            try:
                # Apply the augmentation pipeline
                augmented = augmentations(
                    image=image,
                    bboxes=bboxes,
                    class_ids=class_ids
                )

                # Save the new augmented image
                new_file_name = f"{os.path.splitext(image_info['file_name'])[0]}_aug{i}.jpg"
                new_image_path = os.path.join(AUGMENTED_IMAGE_DIR, new_file_name)
                cv2.imwrite(new_image_path, augmented['image'])

                # Save the new image info
                new_coco_data["images"].append({
                    "id": image_id_counter,
                    "file_name": new_file_name,
                    "width": image_info["width"],
                    "height": image_info["height"]
                })

                # Add the new annotations to the data
                for ann_bbox, ann_class in zip(augmented['bboxes'], augmented['class_ids']):
                    new_coco_data["annotations"].append({
                        "id": annotation_id_counter,
                        "image_id": image_id_counter,
                        "category_id": ann_class,
                        "bbox": ann_bbox,
                        "area": ann_bbox[2] * ann_bbox[3], # w * h
                        "iscrowd": 0
                    })
                    annotation_id_counter += 1

                image_id_counter += 1
            except Exception as e:
                print(f"Error augmenting image {image_info['file_name']}: {e}")

    # Save the new, combined COCO annotation file
    with open(AUGMENTED_ANNOTATION_FILE, 'w') as f:
        json.dump(new_coco_data, f, indent=4)

    print("Dataset augmentation complete.")
    print(f"Original images: {len(coco_data['images'])}")
    print(f"Augmented images: {len(new_coco_data['images']) - len(coco_data['images'])}")
    print(f"Total images in new dataset: {len(new_coco_data['images'])}")
    print(f"New annotation file saved to: {AUGMENTED_ANNOTATION_FILE}")


if _name_ == "_main_":
    augment_dataset()

In [ ]:
import torch
from rfdetr import RFDETRBase
model = RFDETRBase(
    size="base",
    resolution=672,
    gradient_checkpointing=True
)
checkpoint_path = "/workspace/rfdetr_output/checkpoint.pth"

history = model.train(
    dataset_dir="/workspace/dataset/ppe_coco_format",
    output_dir="/workspace/rfdetr_output",
    epochs=10,
    batch_size=8,
    grad_accum_steps=24,
    lr=1e-4,
    warmup_epochs=3,
    lr_scheduler="cosine",
    weight_decay=1e-4,
    optimizer="adamw",
    clip_grad_norm=1.0,
    fp16=True,
    num_workers=4,
    checkpoint_interval=2,
    save_best_model=True,
    tensorboard=True,
    early_stopping=True,
    early_stopping_patience=5,
    resume=checkpoint_path 
)